Чтобы иметь возможность обучать нейронные сети на последовательных данных, необходимо сначала их предварительно обработать. Разобьете данные на пары «входные данные-цель», где входные данные — это некоторое количество последовательных точек данных, а цель — следующая точка данных. Результат: массива NumPy один с входными последовательностями, а другой с соответствующими целями.

In [ ]:
import pandas as pd
import numpy as np

import torch
from torch.utils.data import TensorDataset
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim

!pip install torchmetrics
import torchmetrics

In [84]:
train_data = pd.read_csv('electricity_train.csv', header= 0, sep = ',')
test_data = pd.read_csv('electricity_test.csv', header= 0, sep = ',')

In [85]:
train_data = train_data[:105120]
train_data.shape

(105120, 2)

In [86]:
def create_sequences(df, seq_length):
    xs, ys = [], []
    for i in range(len(df) - seq_length):
        x = df.iloc[i:(i+seq_length), 1]
        y = df.iloc[i+seq_length, 1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

X_train, y_train = create_sequences(train_data, 96) # тут seq_length 96
print(X_train.shape, y_train.shape) # (34944, 96) (34944,)

X_test, y_test = create_sequences(test_data, 96)
# array([[-0.70431852, -0.70431852, -0.67898263, ..., -0.65364675,],...,
#       [ 0.84826447,  0.77200346,  0.72133169, ..., -0.90725895]])
# array([-0.70431852, -0.70431852, -0.65364675, ..., -0.93259484])

dataset_train = TensorDataset(torch.from_numpy(X_train).float(),
                              torch.from_numpy(y_train).float())
dataset_test = TensorDataset(torch.from_numpy(X_test).float(),
                             torch.from_numpy(y_test).float())
# после torch.from_numpy(X_train)
# tensor([[-0.7043, -0.7043, -0.6790,  ..., -0.6536, -0.7299, -0.7043],...
#        [ 0.8483,  0.7720,  0.7213,  ..., -0.9073, -0.9326, -0.9326]], dtype=torch.float64)

#In [8]: dataset_train
#Out[8]: <torch.utils.data.dataset.TensorDataset at 0x7f96bb034610>

(105024, 96) (105024,)


Воспользуемся нашей функцией для создания последовательностей из обучающих данных. Это дает нам почти 35 тысяч обучающих примеров. Чтобы преобразовать их в torch Dataset, мы будем использовать функцию TensorDataset. Передаем два аргумента: входные данные и целевые объекты. Каждый аргумент представляет собой массив NumPy, преобразованный в тензор с помощью torch-dot-from_numpy и проанализированный в float. TensorDataset ведет себя так же, как и все другие наборы данных torch, и его можно передать в DataLoader таким же образом

In [87]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(input_size=1,
                          hidden_size=32,
                          num_layers=2,
                          batch_first=True)
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 32)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

In [88]:
class Net(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1,
                            hidden_size=32,
                            num_layers=2,
                            batch_first=True)
        self.fc = nn.Linear(32, 1)
    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 32)
        c0 = torch.zeros(2, x.size(0), 32)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [89]:
class Net(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.gru = nn.GRU(input_size=1,
                  hidden_size=32,
                  num_layers=2,
                  batch_first=True)
        self.fc = nn.Linear(32, 1)
    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 32)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out

In [90]:
dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=True)

In [91]:
features, labels = next(iter(dataloader_train))
print(f"Features: {features.shape},\nLabels: {labels.shape}")

Features: torch.Size([32, 96]),
Labels: torch.Size([32])


In [92]:
# расширение тензора (конспект)
#for seqs, labels in dataloader_train:
#    print(seqs.shape) #torch.Size([32, 96])
#seqs = seqs.view(32, 96, 1)
#print(seqs.shape) #torch.Size([32, 96, 1])

# сужение
#for seqs, labels in test_loader:
#    print(labels.shape) #torch.Size([32])
#out = net(seqs) # torch.Size([32, 1])
#out = net(seqs).squeeze() # torch.Size([32])

In [93]:
net = Net(1)
net

Net(
  (gru): GRU(1, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)

In [94]:
# цикл обучения
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
for epoch in range(2): #num_epochs 2
    for seqs, labels in dataloader_train:
        seqs = seqs.view(32, 96, 1)
        outputs = net(seqs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#tensor([[-0.9833, -1.1609, -1.2115,  ..., -0.9579, -0.9579,  0.6195],
#        [-1.1609, -1.2115, -1.1862,  ..., -0.9579,  0.6195,  0.6704],
#        [-1.2115, -1.1862, -1.1862,  ...,  0.6195,  0.6704,  0.8992],
#        ...,
#        [-1.2115, -1.1609, -1.1862,  ..., -0.8819,  0.3654,  0.3907]])
#tensor([ 0.6704,  0.8992,  0.7976,  0.5941,  0.4160,  0.4160,  0.4160,  0.2891,
#         0.3398,  0.4670,  0.2891,  0.0816, -0.8819,  0.3654,  0.3907,  0.3907])
# размерности torch.Size([16, 96]) torch.Size([16])
# после torch.Size([16]) torch.Size([16, 96, 1])
#tensor([[[ 1.2805], [ 1.1789], [ 1.1789],..., [-0.5523], [-0.6030], [-0.6536]],
#        [[ 1.1789], [ 1.1789], [ 1.1789],..., [-0.6030], [-0.6536], [-0.6536]],...,
#        [[ 0.8483], [ 0.7720], [ 0.7213],..., [-0.9073], [-0.9326], [-0.9326]]])
# прогнозы
#tensor([[-0.2278], [-0.2283],..., [-0.2300], [-0.2300]], grad_fn=<AddmmBackward0>)
# loss tensor(0.9661, grad_fn=<MseLossBackward0>) (кумулятивное)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [95]:
# цикл оценки
mse = torchmetrics.MeanSquaredError()
net.eval()
with torch.no_grad():
    for seqs, labels in dataloader_test:
        seqs = seqs.view(32, 96, 1)
        outputs = net(seqs).squeeze()
        mse(outputs, labels)
print(f"Test MSE: {mse.compute()}")

Test MSE: 0.6141996383666992
